This was the initial notebook used for the computations of the SL_3 chain complex. Outdated now.

In [2]:
using Pkg
Pkg.activate("../")

  Activating project at `~/Desktop/HigherTSL3/Computations chain complex`


In [4]:
Pkg.add("Combinatorics")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
    Updating `~/Desktop/HigherTSL3/Computations chain complex/Project.toml`
  [861a8166] + Combinatorics v1.0.2
    Updating `~/Desktop/HigherTSL3/Computations chain complex/Manifest.toml`
  [861a8166] + Combinatorics v1.0.2


In [5]:
using LinearAlgebra
using Combinatorics

Computations of SL_3(Z) stabilisers of the cells in the interior of the Voronoi tesselation of the corresponding symmetric space. Also below computations of the "equivariant differentials".

General strategy: Compute the barycenters corresponding to the cells. Check whether they lie in the same orbits. Go back and forth between descriptions via quadratic forms and minimal vectors.

Just the computations I did, would probably need more documentation and clean up. Also, I'm new to Julia, probably that does show.

# The data

In [6]:
e1 = [1,0,0]
e2 = [0,1,0]
e3 = [0,0,1]
e12 = [1,1,0]
e13 = [1,0,1]
e123 = [1,1,1]

3-element Vector{Int64}:
 1
 1
 1

In [7]:
# The following are the different orbits of cells in the interior:
m2 = [e1 e2 e3] # standard
m3_1 = [e1 e2 e3 e12] # 2-additive
m3_2 = [e1 e2 e3 e123] # 3-additive
m4 = [e1 e2 e3 e12 e13] # double-triple
m5 = [e1 e2 e3 e12 e13 e123] # Q
cells = Dict()
cells["m2"] = m2
cells["m3_1"] = m3_1
cells["m3_2"] = m3_2
cells["m4"] = m4
cells["m5"] = m5
cells

Dict{Any, Any} with 5 entries:
  "m2"   => [1 0 0; 0 1 0; 0 0 1]
  "m3_1" => [1 0 0 1; 0 1 0 1; 0 0 1 0]
  "m3_2" => [1 0 0 1; 0 1 0 1; 0 0 1 1]
  "m4"   => [1 0 … 1 1; 0 1 … 1 0; 0 0 … 0 1]
  "m5"   => [1 0 … 1 1; 0 1 … 0 1; 0 0 … 1 1]

In [22]:
# in terms of quadratic forms, they look like this:
standard_cells_quadratic = [m2, m3_1*transpose(m3_1),m3_2*transpose(m3_2),m4*transpose(m4), m5*transpose(m5)]

5-element Vector{Matrix{Int64}}:
 [1 0 0; 0 1 0; 0 0 1]
 [2 1 0; 1 2 0; 0 0 1]
 [2 1 1; 1 2 1; 1 1 2]
 [3 1 1; 1 2 0; 1 0 2]
 [4 2 2; 2 3 1; 2 1 3]

Determinants of the standard cells, seen as quadratic forms:

In [23]:
det_cells = Dict()
det_cells["m2"] = round(Int64,det(m2))
det_cells["m3_1"] = round(Int64,(det(m3_1*transpose(m3_1))))
det_cells["m3_2"] = round(Int64,(det(m3_2*transpose(m3_2))))
det_cells["m4"] = round(Int64,(det(m4*transpose(m4))))
det_cells["m5"] = round(Int64,(det(m5*transpose(m5))))
det_cells

Dict{Any, Any} with 5 entries:
  "m2"   => 1
  "m3_1" => 3
  "m3_2" => 4
  "m4"   => 8
  "m5"   => 16

Note that these determinants already allow one to see which matrix must be in which orbit as congruence with a matrix in SL_n doesn't change the determinant.

# Computations

## Stabilisers

In [24]:
function signed_matrices(dimension)
    # Returns all diagional square "sign matrices" in dimension
    signs_list = []
    for set in powerset(1:dimension)
        matrix = zeros(Int8,dimension,dimension)
        for i in 1:dimension
            if i in set
                matrix[(i-1)*dimension+(i)] = -1
            else
                matrix[(i-1)*dimension+(i)] = 1
            end
        end
        push!(signs_list,matrix)
    end    
    return signs_list
end

signed_matrices (generic function with 1 method)

In [25]:
function bases_in_integer_lattices(matrix)
    # matrix an integer matrix whose columns form a basis of the lattice
    # returns all sets of bases, with all possible sign combinations
    bases = []
    columns = collect(eachcol(matrix)) # set of all columns
    dimension = length(columns[1])
    sign_changes = signed_matrices(dimension)
    for candidate in Combinatorics.permutations(columns,dimension) # all subsets of columns of with dimension-many elements
        matrix_candidate = transpose(reduce(vcat,transpose.(candidate)))
        if (det(matrix_candidate) == 1) || (det(matrix_candidate) == -1)
            #if that's the case, candidate is a basis of Z^dimension
            for sign in sign_changes
                # each basis element can be multiplied by +-1. Add all combinations
                push!(bases, matrix_candidate*sign)
            end
        end
    end
    return bases
end

bases_in_integer_lattices (generic function with 1 method)

In [26]:
bases_in_integer_lattices(m3_1)

144-element Vector{Any}:
 [1 0 0; 0 1 0; 0 0 1]
 [-1 0 0; 0 1 0; 0 0 1]
 [1 0 0; 0 -1 0; 0 0 1]
 [1 0 0; 0 1 0; 0 0 -1]
 [-1 0 0; 0 -1 0; 0 0 1]
 [-1 0 0; 0 1 0; 0 0 -1]
 [1 0 0; 0 -1 0; 0 0 -1]
 [-1 0 0; 0 -1 0; 0 0 -1]
 [1 0 0; 0 0 1; 0 1 0]
 [-1 0 0; 0 0 1; 0 1 0]
 [1 0 0; 0 0 1; 0 -1 0]
 [1 0 0; 0 0 -1; 0 1 0]
 [-1 0 0; 0 0 1; 0 -1 0]
 ⋮
 [-1 0 1; -1 0 0; 0 -1 0]
 [-1 0 -1; -1 0 0; 0 1 0]
 [1 0 -1; 1 0 0; 0 -1 0]
 [-1 0 -1; -1 0 0; 0 -1 0]
 [1 0 0; 1 0 1; 0 1 0]
 [-1 0 0; -1 0 1; 0 1 0]
 [1 0 0; 1 0 1; 0 -1 0]
 [1 0 0; 1 0 -1; 0 1 0]
 [-1 0 0; -1 0 1; 0 -1 0]
 [-1 0 0; -1 0 -1; 0 1 0]
 [1 0 0; 1 0 -1; 0 -1 0]
 [-1 0 0; -1 0 -1; 0 -1 0]

If a g\in SL_3 sends e.g. m3_1 to m3_1, it in particular needs to send the first three of its vertices, given by the identity matrix, to a basis B in m3_1 "modulio +-1" again. I.e. we must have g^t id = B. So all candidate g must be of the form B^t for one of the Bs above.

In [27]:
function stabiliser_coset(matrix1, matrix2)
    #= 
    matrix1, matrix2 integer matrix whose columns form a basis of the lattice. 
    The first columns of matrix1 need to be the standard basis.
    Returns all g \in SL_dimension(Z) that send the quadratic form associated to matrix1 to 
    the one associated to matrix 2
    =#
    elements = []
    quadratic_form1 = matrix1*transpose(matrix1) # the quadratic form associated to the lattice
    quadratic_form2 = matrix2*transpose(matrix2) # the quadratic form associated to the lattice
    candidates = bases_in_integer_lattices(matrix2) # bases in the image
    for g_transpose in candidates
        if det(g_transpose) == 1
            # then it lies in SL
            g = transpose(g_transpose)
            if (g_transpose*quadratic_form1*g == quadratic_form2)
                push!(elements,g)
            end
        end
    end
    return elements
end

stabiliser_coset (generic function with 1 method)

In [28]:
function stabiliser(matrix)
    # matrix an integer matrix whose columns form a basis of the lattice
    stab = stabiliser_coset(matrix,matrix)
    return stab
end

stabiliser (generic function with 1 method)

In [29]:
stab2 = stabiliser(m2)
stab3_1 = stabiliser(m3_1)
stab3_2 = stabiliser(m3_2)
stab4 = stabiliser(m4)
stab5 = stabiliser(m5)
stabilisers_dict = Dict()
stabilisers_dict["m2"] = stab2
stabilisers_dict["m3_1"] = stab3_1
stabilisers_dict["m3_2"] = stab3_2
stabilisers_dict["m4"] = stab4
stabilisers_dict["m5"] = stab5
stabilisers_dict

Dict{Any, Any} with 5 entries:
  "m2"   => Any[[1 0 0; 0 1 0; 0 0 1], [-1 0 0; 0 -1 0; 0 0 1], [-1 0 0; 0 1 0;…
  "m3_1" => Any[[1 0 0; 0 1 0; 0 0 1], [-1 0 0; 0 -1 0; 0 0 1], [-1 0 0; 1 1 0;…
  "m3_2" => Any[[1 0 0; 0 1 0; 0 0 1], [-1 0 0; 0 -1 0; 1 1 1], [-1 0 0; 0 0 -1…
  "m4"   => Any[[1 0 0; 0 1 0; 0 0 1], [-1 0 0; 0 -1 0; 1 0 1], [-1 0 0; 0 0 -1…
  "m5"   => Any[[1 0 0; 0 1 0; 0 0 1], [-1 0 0; 0 0 -1; 0 -1 0], [1 0 0; -1 -1 …

In [30]:
#output to file
open("sl3_stabilisers.txt","w") do io
    for (name, stabiliser) in stabilisers_dict
        println(io,"The stabiliser of the standard cell " * name * " is a group of size "*string(length(stabiliser))*".")
        println(io,"Its elements are:")
        println(io,stabiliser)
        println(io)
    end
end

### Compare to Sage computations
Some checks to see whether the above is correct. It at least agrees with the ones from Sage.

In [31]:
function is_group(list)
    # checks whether a list of matrices is closed under multiplication
    for g in list
        for h in list
            if !(g*h in list)
                return false
            end
        end
    end
    return true
end

is_group (generic function with 1 method)

In [32]:
is_group(stabiliser(m3_1))

true

In [33]:
is_group(stabiliser(m3_2))

true

In [34]:
is_group(stabiliser(m4))

true

In [35]:
is_group(stabiliser(m5))

true

In [36]:
# I copied in the following from the sage computations (using automorphisms() of sage.quadratic_forms.quadratic_form.QuadraticForm)
stab2_sage = [[1 0 0;0 1 0;0 0 1], [1 0 0;0 -1 0;0 0 -1], [0 0 1;1 0 0;0 1 0], [0 0 1;-1 0 0;0 -1 0], [0 1 0;0 0 1;1 0 0], [0 1 0;0 0 -1;-1 0 0], [-1 0 0;0 1 0;0 0 -1], [-1 0 0;0 -1 0;0 0 1], [0 0 -1;1 0 0;0 -1 0], [0 0 -1;-1 0 0;0 1 0], [0 -1 0;0 0 1;-1 0 0], [0 -1 0;0 0 -1;1 0 0], [0 0 -1;0 -1 0;-1 0 0], [0 0 1;0 1 0;-1 0 0], [0 -1 0;-1 0 0;0 0 -1], [0 1 0;1 0 0;0 0 -1], [-1 0 0;0 0 -1;0 -1 0], [1 0 0;0 0 1;0 -1 0], [0 0 1;0 -1 0;1 0 0], [0 0 -1;0 1 0;1 0 0], [0 1 0;-1 0 0;0 0 1], [0 -1 0;1 0 0;0 0 1], [1 0 0;0 0 -1;0 1 0], [-1 0 0;0 0 1;0 1 0]]
stab3_1_sage = [[1 0 0;0 1 0;0 0 1], [1 1 0;-1 0 0;0 0 1], [1 1 0;0 -1 0;0 0 -1], [0 1 0;-1 -1 0;0 0 1], [0 -1 0;-1 0 0;0 0 -1], [1 0 0;-1 -1 0;0 0 -1], [0 -1 0;1 1 0;0 0 1], [0 1 0;1 0 0;0 0 -1], [-1 0 0;1 1 0;0 0 -1], [-1 0 0;0 -1 0;0 0 1], [-1 -1 0;1 0 0;0 0 1], [-1 -1 0;0 1 0;0 0 -1]]
stab3_2_sage = [[1 0 0;0 1 0;0 0 1], [-1 -1 -1;1 0 0;0 0 1], [1 0 0;-1 -1 -1;0 1 0], [0 1 0;0 0 1;1 0 0], [0 0 1;0 1 0;-1 -1 -1], [0 0 1;1 0 0;0 1 0], [0 1 0;-1 -1 -1;0 0 1], [1 0 0;0 0 1;-1 -1 -1], [-1 -1 -1;0 1 0;1 0 0], [0 1 0;1 0 0;-1 -1 -1], [0 0 1;-1 -1 -1;1 0 0], [-1 -1 -1;0 0 1;0 1 0], [1 1 1;0 0 -1;-1 0 0], [-1 0 0;0 -1 0;1 1 1], [1 1 1;-1 0 0;0 -1 0], [-1 0 0;1 1 1;0 0 -1], [0 -1 0;0 0 -1;1 1 1], [0 0 -1;0 -1 0;-1 0 0], [0 0 -1;-1 0 0;1 1 1], [0 -1 0;1 1 1;-1 0 0], [-1 0 0;0 0 -1;0 -1 0], [1 1 1;0 -1 0;0 0 -1], [0 -1 0;-1 0 0;0 0 -1], [0 0 -1;1 1 1;0 -1 0]]
stab4_sage = [[1 0 0;0 1 0;0 0 1], [-1 0 0;1 1 0;0 0 -1], [-1 0 0;0 0 -1;0 -1 0], [1 0 0;0 0 1;-1 -1 0], [1 0 0;-1 -1 0;-1 0 -1], [-1 0 0;0 -1 0;1 0 1], [-1 0 0;1 0 1;1 1 0], [1 0 0;-1 0 -1;0 1 0]]
stab5_sage = [[1 0 0;0 1 0;0 0 1], [1 1 1;0 -1 0;0 0 -1], [0 0 -1;1 0 1;-1 -1 0], [0 -1 0;-1 0 -1;1 1 0], [1 1 0;-1 -1 -1;-1 0 0], [-1 0 -1;1 1 1;1 0 0], [-1 0 0;0 0 -1;0 -1 0], [-1 -1 -1;0 0 1;0 1 0], [0 0 1;1 1 0;-1 0 -1], [0 1 0;-1 -1 0;1 0 1], [-1 -1 0;1 0 0;1 1 1], [1 0 1;-1 0 0;-1 -1 -1], [-1 0 0;1 0 1;1 1 0], [-1 -1 -1;1 1 0;1 0 1], [0 0 1;-1 -1 -1;1 0 0], [0 1 0;1 0 0;-1 -1 -1], [-1 -1 0;0 1 0;0 0 -1], [1 0 1;0 0 -1;0 1 0], [1 0 0;-1 -1 0;-1 0 -1], [1 1 1;-1 0 -1;-1 -1 0], [0 0 -1;-1 0 0;1 1 1], [0 -1 0;1 1 1;-1 0 0], [1 1 0;0 0 1;0 -1 0], [-1 0 -1;0 -1 0;0 0 1]]

24-element Vector{Matrix{Int64}}:
 [1 0 0; 0 1 0; 0 0 1]
 [1 1 1; 0 -1 0; 0 0 -1]
 [0 0 -1; 1 0 1; -1 -1 0]
 [0 -1 0; -1 0 -1; 1 1 0]
 [1 1 0; -1 -1 -1; -1 0 0]
 [-1 0 -1; 1 1 1; 1 0 0]
 [-1 0 0; 0 0 -1; 0 -1 0]
 [-1 -1 -1; 0 0 1; 0 1 0]
 [0 0 1; 1 1 0; -1 0 -1]
 [0 1 0; -1 -1 0; 1 0 1]
 [-1 -1 0; 1 0 0; 1 1 1]
 [1 0 1; -1 0 0; -1 -1 -1]
 [-1 0 0; 1 0 1; 1 1 0]
 [-1 -1 -1; 1 1 0; 1 0 1]
 [0 0 1; -1 -1 -1; 1 0 0]
 [0 1 0; 1 0 0; -1 -1 -1]
 [-1 -1 0; 0 1 0; 0 0 -1]
 [1 0 1; 0 0 -1; 0 1 0]
 [1 0 0; -1 -1 0; -1 0 -1]
 [1 1 1; -1 0 -1; -1 -1 0]
 [0 0 -1; -1 0 0; 1 1 1]
 [0 -1 0; 1 1 1; -1 0 0]
 [1 1 0; 0 0 1; 0 -1 0]
 [-1 0 -1; 0 -1 0; 0 0 1]

In [37]:
Set(stab2) == Set(stab2_sage)

true

In [38]:
Set(stab3_1) == Set(stab3_1_sage)

true

In [39]:
Set(stab3_2) == Set(stab3_2_sage)

true

In [40]:
Set(stab4) == Set(stab4_sage)

true

In [41]:
Set(stab5) == Set(stab5_sage)

true

## Differential

### Old version using quadratic forms

In [42]:
function compute_boundaries_quadratic(matrix)
    boundaries_matrix = []
    for j in 1:size(matrix)[2]
        barycentre = zeros(Int8,3,3)
        for i in 1:size(matrix)[2]
            if j!=i
                barycentre = barycentre+ matrix[:,i]* transpose(matrix[:,i])
            end
        end
        push!(boundaries_matrix, barycentre)
    end
    return boundaries_matrix
end

compute_boundaries_quadratic (generic function with 1 method)

In [43]:
boundaries = []
for matrix in (m3_1,m3_2,m4,m5)
     push!(boundaries,compute_boundaries_quadratic(matrix))
end

Among these boundary matrices, there is one that is not positive definite. Namely the 2-dimensional 2-additive boundary of the 3-dimensional 2-additive matrix:

In [44]:
for i in 1:4
    for matrix in boundaries[i]
        print(isposdef(matrix))
    end
end

truetruefalsetruetruetruetruetruetruetruetruetruetruetruetruetruetruetruetrue

In [45]:
for i in 1:4
    println("Determinant of cells in the boundary:")
    for matrix in boundaries[i]
        println(det(matrix))
    end
end

Determinant of cells in the boundary:
1.0
1.0
0.0
1.0
Determinant of cells in the boundary:
1.0
1.0
1.0
1.0
Determinant of cells in the boundary:
4.0
3.0
3.0000000000000004
3.0
3.0
Determinant of cells in the boundary:
8.0
8.0
8.0
8.0
8.0
8.0


### New version using bases

In [46]:
# a simplified version to check whether everything works:
function compute_boundaries(matrix)
    boundaries_matrix = []
    for j in 1:size(matrix)[2]
        boundary = matrix[begin:end,begin:end .!=j]
        push!(boundaries_matrix, boundary)
    end
    return boundaries_matrix
end

compute_boundaries (generic function with 1 method)

In [47]:
display(m5)
for matrix in compute_boundaries(m5)
    display(matrix)
end

3×6 Matrix{Int64}:
 1  0  0  1  1  1
 0  1  0  1  0  1
 0  0  1  0  1  1

3×5 Matrix{Int64}:
 0  0  1  1  1
 1  0  1  0  1
 0  1  0  1  1

3×5 Matrix{Int64}:
 1  0  1  1  1
 0  0  1  0  1
 0  1  0  1  1

3×5 Matrix{Int64}:
 1  0  1  1  1
 0  1  1  0  1
 0  0  0  1  1

3×5 Matrix{Int64}:
 1  0  0  1  1
 0  1  0  0  1
 0  0  1  1  1

3×5 Matrix{Int64}:
 1  0  0  1  1
 0  1  0  1  1
 0  0  1  0  1

3×5 Matrix{Int64}:
 1  0  0  1  1
 0  1  0  1  0
 0  0  1  0  1

In [48]:
function boundaries_in_group_ring(matrix)
    boundaries_cosets = []
    for j in 1:size(matrix)[2]
        sign = (-1)^(j-1) # the sign of the differential
        boundary = matrix[begin:end,begin:end .!=j] # exclude the j-th column
        boundary_form = boundary*transpose(boundary)
        boundary_det  = round(det(boundary_form)) # use the determinant to identify its orbit 
        for (cell_name, determinant) in det_cells
            # here, there is a unique such cell
            if determinant == boundary_det
                coset = stabiliser_coset(cells[cell_name],boundary)
                push!(boundaries_cosets, (sign, cell_name,coset))
            end
        end
    end
    return boundaries_cosets
end

boundaries_in_group_ring (generic function with 1 method)

In [49]:
bound_m3_1 = boundaries_in_group_ring(m3_1)
bound_m3_2 = boundaries_in_group_ring(m3_2)
bound_m4 = boundaries_in_group_ring(m4)
bound_m5 = boundaries_in_group_ring(m5)
boundaries_dict = Dict()
boundaries_dict["m3_1"]= bound_m3_1
boundaries_dict["m3_2"]= bound_m3_2
boundaries_dict["m4"]= bound_m4
boundaries_dict["m5"]= bound_m5
boundaries_dict

Dict{Any, Any} with 4 entries:
  "m3_1" => Any[(1, "m2", Any[[0 1 0; 0 0 1; 1 1 0], [0 -1 0; 0 0 -1; 1 1 0], […
  "m3_2" => Any[(1, "m2", Any[[0 1 0; 0 0 1; 1 1 1], [0 -1 0; 0 0 -1; 1 1 1], […
  "m4"   => Any[(1, "m3_2", Any[[0 1 0; 0 0 -1; -1 -1 0], [0 -1 0; 0 0 1; -1 0 …
  "m5"   => Any[(1, "m4", Any[[1 1 1; 0 -1 0; 0 0 -1], [-1 -1 -1; 0 1 0; 1 1 0]…

In [50]:
#output to file
open("sl3_boundary_data.txt","w") do io
    for (name, boundary_information) in boundaries_dict
        println(io,"The boundary of " * name * " is a sum of "*string(length(boundary_information))*" elements:")
        println(io)
        for entry in boundary_information
            println(io,"Orbit type of cell in the boundary:")
            println(io,entry[2])
            println(io,"Sign coming from the alternating sum in the differential:")
            println(io,entry[1])
            println(io,"Elements of SL_3 sending the standard cell " * entry[2] * " to this one (i.e. coset of the corresponding stabiliser):")
            println(io,entry[3])
            println(io)
        end
        println(io)
    end
end

#### Some sanity checks

In [51]:
bound_m3_1[2]

(-1, "m2", Any[[-1 0 0; 0 0 1; 1 1 0], [1 0 0; 0 0 -1; 1 1 0], [1 0 0; 0 0 1; -1 -1 0], [-1 0 0; 0 0 -1; -1 -1 0], [1 0 0; 1 1 0; 0 0 1], [-1 0 0; -1 -1 0; 0 0 1], [-1 0 0; 1 1 0; 0 0 -1], [1 0 0; -1 -1 0; 0 0 -1], [0 0 1; 1 0 0; 1 1 0], [0 0 -1; -1 0 0; 1 1 0]  …  [0 0 1; 1 1 0; -1 0 0], [0 0 -1; -1 -1 0; -1 0 0], [-1 -1 0; 1 0 0; 0 0 1], [1 1 0; -1 0 0; 0 0 1], [1 1 0; 1 0 0; 0 0 -1], [-1 -1 0; -1 0 0; 0 0 -1], [1 1 0; 0 0 1; 1 0 0], [-1 -1 0; 0 0 -1; 1 0 0], [-1 -1 0; 0 0 1; -1 0 0], [1 1 0; 0 0 -1; -1 0 0]])

In [52]:
bound_m3_1[3][3]

24-element Vector{Any}:
 [1 0 0; 0 1 0; 0 0 1]
 [-1 0 0; 0 -1 0; 0 0 1]
 [-1 0 0; 0 1 0; 0 0 -1]
 [1 0 0; 0 -1 0; 0 0 -1]
 [-1 0 0; 0 0 1; 0 1 0]
 [1 0 0; 0 0 -1; 0 1 0]
 [1 0 0; 0 0 1; 0 -1 0]
 [-1 0 0; 0 0 -1; 0 -1 0]
 [0 -1 0; 1 0 0; 0 0 1]
 [0 1 0; -1 0 0; 0 0 1]
 [0 1 0; 1 0 0; 0 0 -1]
 [0 -1 0; -1 0 0; 0 0 -1]
 [0 1 0; 0 0 1; 1 0 0]
 [0 -1 0; 0 0 -1; 1 0 0]
 [0 -1 0; 0 0 1; -1 0 0]
 [0 1 0; 0 0 -1; -1 0 0]
 [0 0 1; 1 0 0; 0 1 0]
 [0 0 -1; -1 0 0; 0 1 0]
 [0 0 -1; 1 0 0; 0 -1 0]
 [0 0 1; -1 0 0; 0 -1 0]
 [0 0 -1; 0 1 0; 1 0 0]
 [0 0 1; 0 -1 0; 1 0 0]
 [0 0 1; 0 1 0; -1 0 0]
 [0 0 -1; 0 -1 0; -1 0 0]

In [53]:
Set(bound_m3_1[3][3]) == Set(stab2)

true

In [54]:
# check whether at least all cosets have the correct size
for (sign,name,coset) in bound_m3_1
    print("Orbit of cell in boundary: ")
    print(name)
    print(" Size of coset: ")
    print(length(coset))
    print("\n")
end

Orbit of cell in boundary: m2 Size of coset: 24
Orbit of cell in boundary: m2 Size of coset: 24
Orbit of cell in boundary: m2 Size of coset: 24


In [55]:
# check whether at least all cosets have the correct size
for (sign,name,coset) in bound_m3_2
    print("Orbit of cell in boundary: ")
    print(name)
    print(" Size of coset: ")
    print(length(coset))
    print("\n")
end

Orbit of cell in boundary: m2 Size of coset: 24
Orbit of cell in boundary: m2 Size of coset: 24
Orbit of cell in boundary: m2 Size of coset: 24
Orbit of cell in boundary: m2 Size of coset: 24


In [56]:
# check whether at least all cosets have the correct size
for (sign,name,coset) in bound_m4
    print("Orbit of cell in boundary: ")
    print(name)
    print(" Size of coset: ")
    print(length(coset))
    print("\n")
end

Orbit of cell in boundary: m3_2 Size of coset: 24
Orbit of cell in boundary: m3_1 Size of coset: 12
Orbit of cell in boundary: m3_1 Size of coset: 12
Orbit of cell in boundary: m3_1 Size of coset: 12
Orbit of cell in boundary: m3_1 Size of coset: 12


In [57]:
# check whether at least all cosets have the correct size
for (sign,name,coset) in bound_m5
    print("Orbit of cell in boundary: ")
    print(name)
    print(" Size of coset: ")
    print(length(coset))
    print("\n")
end

Orbit of cell in boundary: m4 Size of coset: 8
Orbit of cell in boundary: m4 Size of coset: 8
Orbit of cell in boundary: m4 Size of coset: 8
Orbit of cell in boundary: m4 Size of coset: 8
Orbit of cell in boundary: m4 Size of coset: 8
Orbit of cell in boundary: m4 Size of coset: 8


In [58]:
bounds = compute_boundaries(m4)
for element in bound_m4[4][3]
    if !(transpose(element)*m3_1*transpose(m3_1)*element==compute_boundaries_quadratic(m4)[4])
        print(false)
    else
        print(true)
    end
end

truetruetruetruetruetruetruetruetruetruetruetrue

In [59]:
bounds = compute_boundaries(m5)
for element in bound_m5[4][3]
    if !(transpose(element)*m4*transpose(m4)*element==compute_boundaries_quadratic(m5)[4])
        print(false)
    else
        print(true)
    end
end

truetruetruetruetruetruetruetrue

# Orientations
Hadn't done this properly before.

In [60]:
function relative_orientation(matrix1,matrix2)
    #= matrix1, matrix2 need to be matrices such that the set of lines spanned by the columns of matrix1
        is the same as that of matrix2
        again, probably more elegant via forms
        computes the orientation of the simplex spanned by the columns
    =# 
    # make signs all positive (could be avoided when working with quadratic forms)
    for x in [matrix1,matrix2]
        for entry in 1:length(x)
            x[entry] = abs(x[entry])
        end
    end
    permutation = Int64[]
    for vector in eachcol(matrix2)
        vertex_index_in_matrix1 = 1
        vector_is_column_of_matrix1 = false
        for vector_standard in eachcol(matrix1)
            if vector == vector_standard
                push!(permutation,vertex_index_in_matrix1)
                vector_is_column_of_matrix1 = true
            end
            vertex_index_in_matrix1 += 1
        end
        vector_is_column_of_matrix1 || error("matrix1 isn't a permutation of matrix2.")
    end
    sign = Combinatorics.levicivita(permutation)
    return sign
end
    

relative_orientation (generic function with 1 method)

In [61]:
# Could surely be done more elegantly (in particular, using quadratic forms would be cleaner)
function orientation_stabiliser(matrix, stabiliser)
    orientation_orbit = []
    # create a list of all ordered simplices coming from matrix under elements of stabiliser
    for g in stabiliser
        push!(orientation_orbit,transpose(g)*matrix)
    end
    signs = []
    for matrix2 in orientation_orbit
        sign = relative_orientation(matrix,matrix2)
        push!(signs,sign)
    end
    return signs
end

orientation_stabiliser (generic function with 1 method)

In [69]:
stab2

24-element Vector{Any}:
 [1 0 0; 0 1 0; 0 0 1]
 [-1 0 0; 0 -1 0; 0 0 1]
 [-1 0 0; 0 1 0; 0 0 -1]
 [1 0 0; 0 -1 0; 0 0 -1]
 [-1 0 0; 0 0 1; 0 1 0]
 [1 0 0; 0 0 -1; 0 1 0]
 [1 0 0; 0 0 1; 0 -1 0]
 [-1 0 0; 0 0 -1; 0 -1 0]
 [0 -1 0; 1 0 0; 0 0 1]
 [0 1 0; -1 0 0; 0 0 1]
 [0 1 0; 1 0 0; 0 0 -1]
 [0 -1 0; -1 0 0; 0 0 -1]
 [0 1 0; 0 0 1; 1 0 0]
 [0 -1 0; 0 0 -1; 1 0 0]
 [0 -1 0; 0 0 1; -1 0 0]
 [0 1 0; 0 0 -1; -1 0 0]
 [0 0 1; 1 0 0; 0 1 0]
 [0 0 -1; -1 0 0; 0 1 0]
 [0 0 -1; 1 0 0; 0 -1 0]
 [0 0 1; -1 0 0; 0 -1 0]
 [0 0 -1; 0 1 0; 1 0 0]
 [0 0 1; 0 -1 0; 1 0 0]
 [0 0 1; 0 1 0; -1 0 0]
 [0 0 -1; 0 -1 0; -1 0 0]

In [68]:
stab3_1

12-element Vector{Any}:
 [1 0 0; 0 1 0; 0 0 1]
 [-1 0 0; 0 -1 0; 0 0 1]
 [-1 0 0; 1 1 0; 0 0 -1]
 [1 0 0; -1 -1 0; 0 0 -1]
 [0 1 0; 1 0 0; 0 0 -1]
 [0 -1 0; -1 0 0; 0 0 -1]
 [0 -1 0; 1 1 0; 0 0 1]
 [0 1 0; -1 -1 0; 0 0 1]
 [-1 -1 0; 1 0 0; 0 0 1]
 [1 1 0; -1 0 0; 0 0 1]
 [-1 -1 0; 0 1 0; 0 0 -1]
 [1 1 0; 0 -1 0; 0 0 -1]

In [70]:
stab3_2

24-element Vector{Any}:
 [1 0 0; 0 1 0; 0 0 1]
 [-1 0 0; 0 -1 0; 1 1 1]
 [-1 0 0; 0 0 -1; 0 -1 0]
 [1 0 0; 0 0 1; -1 -1 -1]
 [1 0 0; -1 -1 -1; 0 1 0]
 [-1 0 0; 1 1 1; 0 0 -1]
 [0 -1 0; -1 0 0; 0 0 -1]
 [0 1 0; 1 0 0; -1 -1 -1]
 [0 1 0; 0 0 1; 1 0 0]
 [0 -1 0; 0 0 -1; 1 1 1]
 [0 -1 0; 1 1 1; -1 0 0]
 [0 1 0; -1 -1 -1; 0 0 1]
 [0 0 1; 1 0 0; 0 1 0]
 [0 0 -1; -1 0 0; 1 1 1]
 [0 0 -1; 0 -1 0; -1 0 0]
 [0 0 1; 0 1 0; -1 -1 -1]
 [0 0 1; -1 -1 -1; 1 0 0]
 [0 0 -1; 1 1 1; 0 -1 0]
 [1 1 1; -1 0 0; 0 -1 0]
 [-1 -1 -1; 1 0 0; 0 0 1]
 [-1 -1 -1; 0 1 0; 1 0 0]
 [1 1 1; 0 -1 0; 0 0 -1]
 [1 1 1; 0 0 -1; -1 0 0]
 [-1 -1 -1; 0 0 1; 0 1 0]

In [71]:
stab4

8-element Vector{Any}:
 [1 0 0; 0 1 0; 0 0 1]
 [-1 0 0; 0 -1 0; 1 0 1]
 [-1 0 0; 0 0 -1; 0 -1 0]
 [1 0 0; 0 0 1; -1 -1 0]
 [-1 0 0; 1 1 0; 0 0 -1]
 [1 0 0; -1 -1 0; -1 0 -1]
 [1 0 0; -1 0 -1; 0 1 0]
 [-1 0 0; 1 0 1; 1 1 0]

In [72]:
stab5

24-element Vector{Any}:
 [1 0 0; 0 1 0; 0 0 1]
 [-1 0 0; 0 0 -1; 0 -1 0]
 [1 0 0; -1 -1 0; -1 0 -1]
 [-1 0 0; 1 0 1; 1 1 0]
 [0 1 0; 1 0 0; -1 -1 -1]
 [0 1 0; -1 -1 0; 1 0 1]
 [0 -1 0; -1 0 -1; 1 1 0]
 [0 -1 0; 1 1 1; -1 0 0]
 [0 0 -1; -1 0 0; 1 1 1]
 [0 0 1; 1 1 0; -1 0 -1]
 [0 0 -1; 1 0 1; -1 -1 0]
 [0 0 1; -1 -1 -1; 1 0 0]
 [-1 -1 0; 1 0 0; 1 1 1]
 [-1 -1 0; 0 1 0; 0 0 -1]
 [1 1 0; 0 0 1; 0 -1 0]
 [1 1 0; -1 -1 -1; -1 0 0]
 [1 0 1; -1 0 0; -1 -1 -1]
 [-1 0 -1; 0 -1 0; 0 0 1]
 [1 0 1; 0 0 -1; 0 1 0]
 [-1 0 -1; 1 1 1; 1 0 0]
 [1 1 1; 0 -1 0; 0 0 -1]
 [-1 -1 -1; 0 0 1; 0 1 0]
 [-1 -1 -1; 1 1 0; 1 0 1]
 [1 1 1; -1 0 -1; -1 -1 0]

The following calculations show that only for m5, every element of the stabiliser preserves the orientation of the simplex.

In [62]:
orientation_stabiliser(m2,stab2)

24-element Vector{Any}:
  1
  1
  1
  1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
  1
  1
  1
  1
  1
  1
  1
  1
 -1
 -1
 -1
 -1

In [51]:
orientation_stabiliser(m3_1,stab3_1)

12-element Vector{Any}:
  1
  1
 -1
 -1
 -1
 -1
  1
  1
  1
  1
 -1
 -1

In [52]:
orientation_stabiliser(m3_2,stab3_2)

24-element Vector{Any}:
  1
 -1
 -1
  1
  1
 -1
 -1
  1
  1
 -1
 -1
  1
  1
 -1
 -1
  1
  1
 -1
 -1
  1
  1
 -1
 -1
  1

In [46]:
orientation_stabiliser(m4,stab4)

8-element Vector{Any}:
  1
 -1
  1
 -1
 -1
  1
 -1
  1

In [47]:
orientation_stabiliser(m5,stab5)

24-element Vector{Any}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

## Boundaries with relative orientations

In [73]:
function stabiliser_coset_with_orientation(matrix1, matrix2)
    #= 
    matrix1, matrix2 integer matrix whose columns form a basis of the lattice. 
    The first columns of matrix1 need to be the standard basis.
    Returns all (g,orientation) such that g\in SL_dimension(Z) sends the quadratic form associated to matrix1 to 
    the one associated to matrix 2 and orientation is the relative orientation of the corresponding simplices
    =#
    elements = []
    quadratic_form1 = matrix1*transpose(matrix1) # the quadratic form associated to the lattice
    quadratic_form2 = matrix2*transpose(matrix2) # the quadratic form associated to the lattice
    candidates = bases_in_integer_lattices(matrix2) # bases in the image
    for g_transpose in candidates
        if det(g_transpose) == 1
            # then it lies in SL
            g = transpose(g_transpose)
            if (g_transpose*quadratic_form1*g == quadratic_form2)
                orientation = relative_orientation(g_transpose*matrix1,matrix2)
                # this is the relative orientation of g.standard cell compared to the boundary
                push!(elements, (g,orientation))
            end
        end
    end
    return elements
end

stabiliser_coset_with_orientation (generic function with 1 method)

In [92]:
function boundaries_in_group_ring_with_orientation(matrix)
    boundaries_cosets = []
    for j in 1:size(matrix)[2]
        sign = (-1)^(j-1) # the sign of the differential
        boundary = matrix[begin:end,begin:end .!=j] # exclude the j-th column
        boundary_form = boundary*transpose(boundary)
        boundary_det  = round(det(boundary_form)) # use the determinant to identify its orbit 
        for (cell_name, determinant) in det_cells
            # here, there is a unique such cell
            if determinant == boundary_det
                coset_with_orientation = stabiliser_coset_with_orientation(cells[cell_name],boundary)
                @info coset_with_orientation
                push!(boundaries_cosets, (sign, cell_name, coset_with_orientation))
            end
        end
    end
    return boundaries_cosets
end

boundaries_in_group_ring_with_orientation (generic function with 1 method)

In [75]:
bound_m3_1 = boundaries_in_group_ring_with_orientation(m3_1)
bound_m3_2 = boundaries_in_group_ring_with_orientation(m3_2)
bound_m4 = boundaries_in_group_ring_with_orientation(m4)
bound_m5 = boundaries_in_group_ring_with_orientation(m5)
boundaries_dict = Dict()
boundaries_dict["m3_1"]= bound_m3_1
boundaries_dict["m3_2"]= bound_m3_2
boundaries_dict["m4"]= bound_m4
boundaries_dict["m5"]= bound_m5
boundaries_dict

Dict{Any, Any} with 4 entries:
  "m3_1" => Any[(1, "m2", Any[([0 1 0; 0 0 1; 1 1 0], 1), ([0 -1 0; 0 0 -1; 1 1…
  "m3_2" => Any[(1, "m2", Any[([0 1 0; 0 0 1; 1 1 1], 1), ([0 -1 0; 0 0 -1; 1 1…
  "m4"   => Any[(1, "m3_2", Any[([0 1 0; 0 0 -1; -1 -1 0], 1), ([0 -1 0; 0 0 1;…
  "m5"   => Any[(1, "m4", Any[([1 1 1; 0 -1 0; 0 0 -1], -1), ([-1 -1 -1; 0 1 0;…

In [143]:
bound_m5

6-element Vector{Any}:
 (1, "m4", Any[([1 1 1; 0 -1 0; 0 0 -1], -1), ([-1 -1 -1; 0 1 0; 1 1 0], 1), ([-1 -1 -1; 0 0 1; 0 1 0], -1), ([1 1 1; 0 0 -1; -1 0 -1], 1), ([1 1 1; -1 -1 0; 0 -1 0], 1), ([-1 -1 -1; 1 1 0; 1 0 1], -1), ([-1 -1 -1; 1 0 1; 0 0 1], 1), ([1 1 1; -1 0 -1; -1 -1 0], -1)])
 (-1, "m4", Any[([0 0 1; 1 0 0; 1 1 0], -1), ([0 0 -1; -1 0 0; 1 1 1], 1), ([0 0 -1; -1 -1 0; -1 0 0], -1), ([0 0 1; 1 1 0; -1 0 -1], 1), ([0 0 -1; 1 0 1; -1 -1 0], 1), ([0 0 1; -1 0 -1; -1 -1 -1], -1), ([0 0 1; -1 -1 -1; 1 0 0], 1), ([0 0 -1; 1 1 1; 1 0 1], -1)])
 (1, "m4", Any[([0 -1 0; -1 0 0; -1 0 -1], 1), ([0 1 0; 1 0 0; -1 -1 -1], -1), ([0 1 0; -1 -1 0; 1 0 1], -1), ([0 -1 0; 1 1 0; 1 1 1], 1), ([0 1 0; 1 0 1; 1 0 0], 1), ([0 -1 0; -1 0 -1; 1 1 0], -1), ([0 -1 0; 1 1 1; -1 0 0], -1), ([0 1 0; -1 -1 -1; -1 -1 0], 1)])
 (-1, "m4", Any[([-1 0 -1; 1 0 0; 0 -1 0], -1), ([1 0 1; -1 0 0; -1 -1 -1], 1), ([1 0 1; 0 1 0; -1 0 0], -1), ([-1 0 -1; 0 -1 0; 0 0 1], 1), ([1 0 1; 0 0 -1; 0 1 0], 1), ([-1 0 -1;

In [90]:
stab2 = stabiliser_coset_with_orientation(m2,m2)
stab3_1 = stabiliser_coset_with_orientation(m3_1,m3_1)
stab3_2 = stabiliser_coset_with_orientation(m3_2,m3_2)
stab4 = stabiliser_coset_with_orientation(m4,m4)
stab5 = stabiliser_coset_with_orientation(m5,m5)
stabilisers_dict = Dict()
stabilisers_dict["m2"] = stab2
stabilisers_dict["m3_1"] = stab3_1
stabilisers_dict["m3_2"] = stab3_2
stabilisers_dict["m4"] = stab4
stabilisers_dict["m5"] = stab5
stabilisers_dict

Dict{Any, Any} with 5 entries:
  "m2"   => Any[([1 0 0; 0 1 0; 0 0 1], 1), ([-1 0 0; 0 -1 0; 0 0 1], 1), ([-1 …
  "m3_1" => Any[([1 0 0; 0 1 0; 0 0 1], 1), ([-1 0 0; 0 -1 0; 0 0 1], 1), ([-1 …
  "m3_2" => Any[([1 0 0; 0 1 0; 0 0 1], 1), ([-1 0 0; 0 -1 0; 1 1 1], -1), ([-1…
  "m4"   => Any[([1 0 0; 0 1 0; 0 0 1], 1), ([-1 0 0; 0 -1 0; 1 0 1], -1), ([-1…
  "m5"   => Any[([1 0 0; 0 1 0; 0 0 1], 1), ([-1 0 0; 0 0 -1; 0 -1 0], 1), ([1 …

In [101]:
stabilisers_dict["m5"]

24-element Vector{Any}:
 ([1 0 0; 0 1 0; 0 0 1], 1)
 ([-1 0 0; 0 0 -1; 0 -1 0], 1)
 ([1 0 0; -1 -1 0; -1 0 -1], 1)
 ([-1 0 0; 1 0 1; 1 1 0], 1)
 ([0 1 0; 1 0 0; -1 -1 -1], 1)
 ([0 1 0; -1 -1 0; 1 0 1], 1)
 ([0 -1 0; -1 0 -1; 1 1 0], 1)
 ([0 -1 0; 1 1 1; -1 0 0], 1)
 ([0 0 -1; -1 0 0; 1 1 1], 1)
 ([0 0 1; 1 1 0; -1 0 -1], 1)
 ([0 0 -1; 1 0 1; -1 -1 0], 1)
 ([0 0 1; -1 -1 -1; 1 0 0], 1)
 ([-1 -1 0; 1 0 0; 1 1 1], 1)
 ([-1 -1 0; 0 1 0; 0 0 -1], 1)
 ([1 1 0; 0 0 1; 0 -1 0], 1)
 ([1 1 0; -1 -1 -1; -1 0 0], 1)
 ([1 0 1; -1 0 0; -1 -1 -1], 1)
 ([-1 0 -1; 0 -1 0; 0 0 1], 1)
 ([1 0 1; 0 0 -1; 0 1 0], 1)
 ([-1 0 -1; 1 1 1; 1 0 0], 1)
 ([1 1 1; 0 -1 0; 0 0 -1], 1)
 ([-1 -1 -1; 0 0 1; 0 1 0], 1)
 ([-1 -1 -1; 1 1 0; 1 0 1], 1)
 ([1 1 1; -1 0 -1; -1 -1 0], 1)